## Submission instructions

All code that you write should be in this notebook. Please include your names and student numbers. You have to submit this notebook, with your code and answers filled in. Make sure to add enough documentation.

For questions, make use of the "Lab" session (see schedule).
Questions can also be posted to the MS teams channel called "Lab".

**Note:** You are free to make use of Python libraries (e.g., numpy, sklearn, etc.) except any *fairness* libraries.

#### Name and student numbers
Bar Melinarskiy - 2482975

Julia Baas - 6082826

## Dataset

In this assignment we are going to use the **COMPAS** dataset.

If you haven't done so already, take a look at this article: https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing.
For background on the dataset, see https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm.

**Reading in the COMPAS dataset**

The dataset can be downloaded here: https://github.com/propublica/compas-analysis/blob/master/compas-scores-two-years.csv

For this assignment, we focus on the protected attribute *race*.

The label (the variable we want to be able to predict) represents recidivism, which is defined as a new arrest within 2 years.

In [175]:
!wget -c https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [176]:
import pandas as pd
compas_data = pd.read_csv('compas-scores-two-years.csv')

We apply several data preprocessing steps, including only retaining Caucasians and African Americans.

In [177]:
compas_data = compas_data[(compas_data.days_b_screening_arrest <= 30)
            & (compas_data.days_b_screening_arrest >= -30)
            & (compas_data.is_recid != -1)
            & (compas_data.c_charge_degree != 'O')
            & (compas_data.score_text != 'N/A')
            & ((compas_data.race == 'Caucasian') | (compas_data.race == 'African-American'))]

Take a look at the data:

In [178]:
print(compas_data.head())

    id              name      first    last compas_screening_date     sex  \
1    3       kevon dixon      kevon   dixon            2013-01-27    Male   
2    4          ed philo         ed   philo            2013-04-14    Male   
6    8     edward riddle     edward  riddle            2014-02-19    Male   
8   10  elizabeth thieme  elizabeth  thieme            2014-03-16  Female   
10  14    benjamin franc   benjamin   franc            2013-11-26    Male   

           dob  age       age_cat              race  ...  v_decile_score  \
1   1982-01-22   34       25 - 45  African-American  ...               1   
2   1991-05-14   24  Less than 25  African-American  ...               3   
6   1974-07-23   41       25 - 45         Caucasian  ...               2   
8   1976-06-03   39       25 - 45         Caucasian  ...               1   
10  1988-06-01   27       25 - 45         Caucasian  ...               4   

    v_score_text  v_screening_date  in_custody  out_custody  priors_count.1  \
1

Now take a look at the distribution of the protected attribute `race` and the distribution of our outcome variable `two_year_recid`.

**Note:** in the context of fair machine learning, the favorable label here is no recidivism, i.e., ```two_year_recid = 0```. So think about how what you will code as the positive class in your machine learning experiments, and make sure your interpretation of the results is consistent with this.

In [179]:
print('Number of instances per race category:')
print(compas_data[['race', 'two_year_recid']].value_counts())

Number of instances per race category:
race              two_year_recid
African-American  1                 1661
                  0                 1514
Caucasian         0                 1281
                  1                  822
Name: count, dtype: int64


## Data analysis

### **1. Exploration**

First we perform an exploratory analysis of the data.

**Question:** What is the size of the data? (i.e. how many data instances does it contain?)


In [180]:
# Your code
print(f"Dataset shape: {compas_data.shape}")
dataset_size = len(compas_data)
print(f"Number of instances: {dataset_size}")
print(f"Number of features: {len(compas_data.columns)}")
print(f"Number of classes: {len(compas_data['two_year_recid'].unique())}")

print("Overall class distribution:")
distribution = compas_data['two_year_recid'].value_counts().reset_index()
distribution.columns = ['two_year_recid', 'Count']
distribution['Percentage'] = (distribution['Count'] / compas_data['two_year_recid'].count()) * 100
print(distribution.to_string(index=False))

Dataset shape: (5278, 53)
Number of instances: 5278
Number of features: 53
Number of classes: 2
Overall class distribution:
 two_year_recid  Count  Percentage
              0   2795   52.955665
              1   2483   47.044335


***Answer:*** 
We have 5,278 instances in the data

**Question:** In the dataset, the protected attribute is `race`, which has two categories: White and African Americans. How many data instances belong to each category?

In [181]:
# Your code
print("Overall race distribution:")
feature = "race"
distribution = compas_data[feature].value_counts().reset_index()
distribution.columns = [feature, 'Count']
distribution['Percentage'] = (distribution['Count'] / compas_data[feature].count()) * 100
print(distribution.to_string(index=False))

Overall race distribution:
            race  Count  Percentage
African-American   3175   60.155362
       Caucasian   2103   39.844638


**Question:** What are the base rates (the probability of a favorable outcome for the two protected attribute classes)?

In [182]:
# Your code
protected_feature = "race"
label = "two_year_recid"

# Group by the protected feature and calculate the counts and proportions for each label value
counts = compas_data.groupby(protected_feature)[label].value_counts(normalize=False).unstack().fillna(0)
proportions = compas_data.groupby(protected_feature)[label].value_counts(normalize=True).unstack().fillna(0)

# Reset the index for better readability
counts = counts.reset_index()
proportions = proportions.reset_index()

# Rename the columns for clarity
counts.columns = [protected_feature] + [f"Count_{label}_{i}" for i in counts.columns[1:]]
proportions.columns = [protected_feature] + [f"Probability_{label}_{i}" for i in proportions.columns[1:]]

# Combine counts and proportions into a single DataFrame
distribution = pd.merge(counts, proportions, on=protected_feature)

# Print the combined DataFrame without the index
print("\nDistribution of the label (two_year_recid) by the protected feature (race):")
print(distribution.to_string(index=False))

# Explicitly print the base rates and counts for the favorable outcome (two_year_recid = 0)
print("\nSO the base rates (two_year_recid = 0) and counts are:")
for _, row in distribution.iterrows():
    print(f"For {row[protected_feature]}: Count={row[f'Count_{label}_0']}, Probability={row[f'Probability_{label}_0']:.3f}")


Distribution of the label (two_year_recid) by the protected feature (race):
            race  Count_two_year_recid_0  Count_two_year_recid_1  Probability_two_year_recid_0  Probability_two_year_recid_1
African-American                    1514                    1661                       0.47685                       0.52315
       Caucasian                    1281                     822                       0.60913                       0.39087

SO the base rates (two_year_recid = 0) and counts are:
For African-American: Count=1514, Probability=0.477
For Caucasian: Count=1281, Probability=0.609


**Question:** What are the base rates for the combination of both race and sex categories?

In [183]:
# Your code
protected_features = ["race", "sex"]
label = "two_year_recid"
      
# Group by the protected features and calculate the counts and proportions for each label value
distribution = compas_data.groupby(protected_features)[label].value_counts(normalize=False).unstack().fillna(0)
proportions = compas_data.groupby(protected_features)[label].value_counts(normalize=True).unstack().fillna(0)

# Combine counts and proportions into a single DataFrame
distribution = distribution.reset_index()
proportions = proportions.reset_index()
distribution.columns = protected_features + [f"Count_{label}_{i}" for i in distribution.columns[len(protected_features):]]
proportions.columns = protected_features + [f"Probability_{label}_{i}" for i in proportions.columns[len(protected_features):]]
combined = pd.merge(distribution, proportions, on=protected_features)

# Print the combined DataFrame without the index
print("\nDistribution of the label (two_year_recid) by the protected features (race, sex):")

print(combined.to_string(index=False))

# Explicitly print the base rates and counts for the favorable outcome (two_year_recid = 0)
print("\nSO the base rates (two_year_recid = 0) and counts are:")
for _, row in combined.iterrows():
    feature_combination = ", ".join([f"{feature}={row[feature]}" for feature in protected_features])
    print(f"For {feature_combination}: Count={row[f'Count_{label}_0']}, Probability={row[f'Probability_{label}_0']:.3f}")


Distribution of the label (two_year_recid) by the protected features (race, sex):
            race    sex  Count_two_year_recid_0  Count_two_year_recid_1  Probability_two_year_recid_0  Probability_two_year_recid_1
African-American Female                     346                     203                      0.630237                      0.369763
African-American   Male                    1168                    1458                      0.444783                      0.555217
       Caucasian Female                     312                     170                      0.647303                      0.352697
       Caucasian   Male                     969                     652                      0.597779                      0.402221

SO the base rates (two_year_recid = 0) and counts are:
For race=African-American, sex=Female: Count=346, Probability=0.630
For race=African-American, sex=Male: Count=1168, Probability=0.445
For race=Caucasian, sex=Female: Count=312, Probability=0.647
For r

**Question**

Write down a short interpretation of the statistics you calculated. What do you see?
> **Answer: Female offenders are less likely to become a recidivist than male offenders (0.630 over 0.445 and 0.647 over 0.598 for African-American and Caucasion respectively). Furthermore, African-American offenders are more likely to become a recidivist than Caucasian offenders (0.445 over 0.598 and 0.630 over 0.647 for male and female respectively).**

### **2. Performance measures**

You will have to measure the performance and fairness of different classifiers in question 5. The performance will be calculated with the precision, recall, F1 and accuracy.
Additionally, you will have to calculate the statistical/demographic parity, the true positive rate (recall) and false positive rate per race group.

Make sure that you are able to calculate these metrics in the cell below.

In [184]:
# Your code for the performance measures
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

favorable_outcome = 0  # Define the favorable outcome (e.g., 0 for 'not recidivist')
unfavorable_outcome = 1  # Define the unfavorable outcome (e.g., 1 for 'recidivist')

# Function to calculate overall performance metrics
def calculate_performance_metrics(y_true, y_pred):
    metrics = {
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall (TPR)": recall_score(y_true, y_pred, zero_division=0),
        "F1 Score": f1_score(y_true, y_pred, zero_division=0),
        "Accuracy": accuracy_score(y_true, y_pred)
    }
    df = pd.DataFrame.from_dict(metrics, orient='index', columns=['Value'])
    return df

# Function to calculate statistical/demographic parity
def calculate_statistical_parity(x, y_pred, protected_attribute):
    parity = {}
    y_groups = {g: y_pred.loc[df.index] for g, df in x.groupby(protected_attribute)}


    for key, group in y_groups.items():
        favorable_rate = np.mean( group == favorable_outcome)  # Favorable outcome is `0`
        parity[key] = favorable_rate
    df = pd.DataFrame.from_dict(parity, orient='index')
    df.columns = ['Favorable Outcome Rate']
    df.index = df.index.map(lambda x: 'Caucasian' if x == 1 else 'African-American')
    
    return df

# Function to calculate TPR and FPR per group
def calculate_group_metrics(X, y_true, y_pred, protected_attribute):
    group_metrics = {}
    # Group the data by the protected attribute    
    X_groups = {g: df.drop(columns=protected_attribute) for g, df in X.groupby(protected_attribute)}
    
    # Use the indices from the grouped X to split y_true and y_pred
    y_true_groups = {g: y_true.loc[df.index] for g, df in X_groups.items()}
    y_pred_groups = {g: y_pred.loc[df.index] for g, df in X_groups.items()}


    for group in X_groups.keys():
        # Get the corresponding y_true and y_pred groups
        y_true_group = y_true_groups[group]
        y_pred_group = y_pred_groups[group]
        # True Positive Rate (TPR)
        tpr = recall_score(y_true_group, y_pred_group, zero_division=0)
        
        # False Positive Rate (FPR)
        fp = np.sum((y_pred_group == unfavorable_outcome) & (y_true_group == favorable_outcome))
        tn = np.sum((y_pred_group == favorable_outcome) & (y_true_group == favorable_outcome))
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
        
        group_metrics[group] = {"TPR": tpr, "FPR": fpr}

    # Convert the dictionary to a DataFrame
    group_metrics_df = pd.DataFrame.from_dict(group_metrics, orient='index')
    
    # Set the index to meaningful labels
    group_metrics_df.index = group_metrics_df.index.map(lambda x: 'Caucasian' if x == 1 else 'African-American')

    return group_metrics_df

### **3. Prepare the data**
For the classifiers in question 5, the input of the model can only contain numerical values, it is therefore important to convert the strings in the columns (features) of interest of the `compas_data` to floats or integers.

The columns of interest are features that you think will be informative or interesting in predicting the outcome variable.
Use the cell below to explore which of the Compas variables you need to convert to be able to use them for the classifiers.

Generate a new dataframe with your selected features in the right encoding (also make sure to include `two_year_recid`). You can implement this yourself, or use the `LabelEncoder` from `sklearn`.

**Note:** you do not need to convert all columns/features, only the ones you are interested in. However, do **not** include the feature `is_recid`.

In [185]:
print(compas_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5278 entries, 1 to 7212
Data columns (total 53 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       5278 non-null   int64  
 1   name                     5278 non-null   object 
 2   first                    5278 non-null   object 
 3   last                     5278 non-null   object 
 4   compas_screening_date    5278 non-null   object 
 5   sex                      5278 non-null   object 
 6   dob                      5278 non-null   object 
 7   age                      5278 non-null   int64  
 8   age_cat                  5278 non-null   object 
 9   race                     5278 non-null   object 
 10  juv_fel_count            5278 non-null   int64  
 11  decile_score             5278 non-null   int64  
 12  juv_misd_count           5278 non-null   int64  
 13  juv_other_count          5278 non-null   int64  
 14  priors_count             5278

In [186]:
#filtering only the features we find interesting:
columns_to_keep = ["sex", "age", "race", "priors_count", "juv_fel_count", "two_year_recid"]
new_dataset = compas_data[columns_to_keep].copy()

#Converting sex values into 0 and 1
new_dataset["sex"] = (new_dataset["sex"] == 'Male').astype(int) #Male = 1, female = 0
new_dataset["race"] = (new_dataset["race"] == 'Caucasian').astype(int) #Caucasian = 1, A-A = 0

#check it:
print(new_dataset.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5278 entries, 1 to 7212
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   sex             5278 non-null   int32
 1   age             5278 non-null   int64
 2   race            5278 non-null   int32
 3   priors_count    5278 non-null   int64
 4   juv_fel_count   5278 non-null   int64
 5   two_year_recid  5278 non-null   int64
dtypes: int32(2), int64(4)
memory usage: 247.4 KB
None


**Question**

Give a short motivation (one-two sentence) per feature why you think this is informative or interesting to take into account.
> Answer: Sex is important to include because we saw that the reoffender scores differed, so it could be an indicator. Furthermore, the same holds for race and this was an important topic in the discussion on the COMPAS system. We also wanted to include age because young offenders have more time and are more likely to reoffend. This was also pointed out by ProPublica. The amount of prior offenses can also be a good indicator, since the offender already is a reoffender. The same logic holds for juvenile felony counts. 

### **4. Train and test split**

Divide the dataset into a train (80%) and test split (20%), either by implementing it yourself, or by using an existing library.

**Note:** Usually when carrying out machine learning experiments,
we also need a dev set for developing and selecting our models (incl. tuning of hyper-parameters).
However, in this assignment, the goal is not to optimize
the performance of models so we'll only use a train and test split.




In [187]:
# Your code to split the data
from sklearn.model_selection import train_test_split

# Define the features (X) and the target variable (y)
label = "two_year_recid"
X = new_dataset.drop(columns=[label])  # Drop the target column
y = new_dataset[label]  # Target column

# Split the dataset into training (80%) and testing (20%) setsm 
# using stratified sampling to maintain the distribution of the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Print the sizes of the splits
print(f"Training set size: {X_train.shape[0]} instances")
print(f"Testing set size: {X_test.shape[0]} instances")


Training set size: 4222 instances
Testing set size: 1056 instances


### **5. Classifiers**

Now, train and test different classifiers and report the following statistics:

* Overall performance:

  * Precision
  * Recall
  * F1
  * Accuracy

* Fairness performance:

  * The statistical parity difference for the protected attribute `race`(i.e. the difference in the probability of receiving a favorable label between the two protected attribute groups);
  * The true positive rates of the two protected attribute groups
  * The false positive rates of the two protected attribute groups.

For training the classifier we recommend using scikit-learn (https://scikit-learn.org/stable/).

#### **5.1 Regular classification**
Train a logistic regression classifier with the race feature and all other features that you are interested in.

In [188]:
# Your code for classifier 1
import pprint
from sklearn.linear_model import LogisticRegression

# train the logistic regression model
clf1 = LogisticRegression(random_state=0).fit(X_train, y_train)

# make predictions on the test set
y_pred = pd.Series(clf1.predict(X_test))

# Reset the indices of y_test and y_pred to match the original test dateset DataFrame
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
y_pred = y_pred.reset_index(drop=True)

print("Logstic Regression with all chosen features")
print("\n****************************************************************")
print("The overall scores are:")
df_1 = calculate_performance_metrics(y_test, y_pred)
print(df_1.to_string())

print("\n****************************************************************")
print("The statistical parity difference for race is:")
df_2 = calculate_statistical_parity(X_test, y_pred, "race")
pprint.pprint(df_2)

print("\n****************************************************************")
print("The TPR and FPR of the two protected attribute groups:")
df_3 = calculate_group_metrics(X_test, y_test, y_pred, "race")
# df_2.index = df_2.index.map(lambda x: 'Caucasian' if x == 1 else 'African-American')
pprint.pprint(df_3)

Logstic Regression with all chosen features

****************************************************************
The overall scores are:
                 Value
Precision     0.661327
Recall (TPR)  0.581489
F1 Score      0.618844
Accuracy      0.662879

****************************************************************
The statistical parity difference for race is:
                  Favorable Outcome Rate
African-American                0.472089
Caucasian                       0.752914

****************************************************************
The TPR and FPR of the two protected attribute groups:
                       TPR       FPR
African-American  0.678899  0.363333
Caucasian         0.394118  0.150579


#### **5.2 Without the protected attribute**
Train a logistic regression classifier without the race feature, but with all other features you used in 5.1.


In [189]:
# Your code for classifier 2
import pprint
from sklearn.linear_model import LogisticRegression

#filtering the data without race:
protected_features = ['race']
X_without_protected = X_train.drop(columns=protected_features)
X_test_without_protected = X_test.drop(columns=protected_features)

# train the logistic regression model
clf2 = LogisticRegression(random_state=0).fit(X_without_protected, y_train)

# make predictions on the test set
y_pred_without = pd.Series(clf2.predict(X_test_without_protected))

# Reset the indices of y_test and y_pred to match the original test dateset DataFrame
X_test_without_protected = X_test_without_protected.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
y_pred_without = y_pred_without.reset_index(drop=True)

print("Logstic Regression without protected attribute (race)")
print("\n****************************************************************")
print("The overall scores are:")
df_1 = calculate_performance_metrics(y_test, y_pred_without)
print(df_1.to_string())

print("\n****************************************************************")
print("The statistical parity difference for race is:")
df_2 = calculate_statistical_parity(X_test, y_pred_without, "race")
pprint.pprint(df_2)

print("\n****************************************************************")
print("The TPR and FPR of the two protected attribute groups:")
df_3 = calculate_group_metrics(X_test, y_test, y_pred_without, "race")
# df_2.index = df_2.index.map(lambda x: 'Caucasian' if x == 1 else 'African-American')
pprint.pprint(df_3)

Logstic Regression without protected attribute (race)

****************************************************************
The overall scores are:
                 Value
Precision     0.657658
Recall (TPR)  0.587525
F1 Score      0.620616
Accuracy      0.661932

****************************************************************
The statistical parity difference for race is:
                  Favorable Outcome Rate
African-American                0.473684
Caucasian                       0.734266

****************************************************************
The TPR and FPR of the two protected attribute groups:
                       TPR       FPR
African-American  0.675841  0.363333
Caucasian         0.417647  0.166023


**Question**

Write down a short interpretation of the results you calculated. What do you see?
> Answer: The result is very similar. This means that adding the race doesn't make the system stronger. This is because race is not a good indicator for recidivism, the other features are stronger indicators. The outcome relies mostly on the other features.

#### **5.3 Pre-processing: Reweighing**
Train and test a classifier with weights (see lecture slide for the weight calculation)

In [190]:
# Your code for classifier 3


**Question**

 Report the 4 weights that are used for reweighing and a short **interpretation/discussion** of the weights and the classifier results.
> Answer:


#### **5.4 Post-processing: Equalized odds**
Use the predictions by the first classifier for this post processing part (see lecture slides for more information about post processing for equalized odds).

We have the following parameters (A indicates group membership, Y_{hat} the original prediction, Y_{tilde} the prediction of the derived predictor).

* `p_00` = P(Y_{tilde} = 1 | Y_{hat} = 0 & A = 0)
* `p_01` = P(Y_{tilde} = 1 | Y_{hat} = 0 & A = 1)
* `p_10` = P(Y_{tilde} = 1 | Y_{hat} = 1 & A = 0)
* `p_11` = P(Y_{tilde} = 1 | Y_{hat} = 1 & A = 1)


Normally, the best parameters `p_00, p_01, p_10, p_11` are found with a linear program that minimizes loss between predictions of a derived predictor and the actual labels. In this assignment we will not ask you to do this. Instead, we would like you to follow the next steps to find parameters, post-process the data and check the performance of this classifier with post-processing:

1. Generate 5000 different samples of these 4 parameters randomly;
2. Write a function (or more) that applies these 4 parameters to postprocess the predictions.
3. For each generated set of 4 parameters:
  - Change the predicted labels with the function(s) from step 2;
  - Evaluate these 'new' predictions, by calculating group-wise TPR and FPR, as well as overall performance based on F1 and/or accuracy.
4. Choose the best set of parameters. Take into account the equalized odds fairness measure, as well a performance measure like accuracy or F1.
5. Check the overall performance (precision, recall, accuracy, F1, etc.) of the new predictions after post-processing.

In [ ]:
# Your code for step 1
import random
random.seed(42)

# predictions for the first model
y_pred = pd.Series(clf1.predict(X_test))

# Generate random parameters for the model
random_parameters = []
for _ in range(5000):
  p_00 = random.uniform(0, 1)
  p_01 = random.uniform(0, 1)
  p_10 = random.uniform(0, 1)
  p_11 = random.uniform(0, 1)
  random_parameters.append({(0, 0): p_00,
                            (0, 1): p_01,
                            (1, 0): p_10,
                            (1, 1): p_11})

# Example, first set of random parameters
print(random_parameters[0])

{(0, 0): 0.5714025946899135, (0, 1): 0.4288890546751146, (1, 0): 0.5780913011344704, (1, 1): 0.20609823213950174}


In [ ]:
# Your code for step 2
# Create a dataframe with the necessary information
df_post_data = pd.DataFrame({'race_num': X_test['race'],
                             'pred_labels': y_pred,
                             'true_labels': y_test})

# the number of cases falling in each condition
subset_sizes = {
    (0, 0): len(df_post_data.query('pred_labels == 0 & race_num == 0')),
    (0, 1): len(df_post_data.query('pred_labels == 0 & race_num == 1')),
    (1, 0): len(df_post_data.query('pred_labels == 1 & race_num == 0')),
    (1, 1): len(df_post_data.query('pred_labels == 1 & race_num == 1'))

}

def generate_labels(subset_sizes, p_dict):
    """
    subset_sizes: dict with number of cases falling in each condition
    p_dict: the postprocessing parameters
    """
    new_predictions = {}

    for (prediction, group), p in p_dict.items():

      # The number of instances for which we need to generate labels
      num_instances = subset_sizes[(prediction, group)]
      
      # Write your code here.
      # Get labels and prediction of the current subgroup
      y_tilde_for_subgroup = df_post_data.query(f'pred_labels == {prediction} & race_num == {group}')['pred_labels'].values

      # flip the labels according to the postprocessing parameters
      flip_mask = np.random.rand(num_instances) < p
      y_tilde_for_subgroup_new = y_tilde_for_subgroup * (1 - flip_mask) + np.abs(y_tilde_for_subgroup-1) * flip_mask
      # save the new predictions
      new_predictions[(prediction, group)] = y_tilde_for_subgroup_new

    return new_predictions

In [193]:
# Your code for step 3

for p_dict in random_parameters:

  new_predictions = generate_labels(subset_sizes, p_dict)

  # replace the predictions
  df_copy = df_post_data.copy()

  for (pred, group), p in p_dict.items():

    new_preds = new_predictions[(pred,group)]
    df_copy.loc[(df_post_data['pred_labels'] == pred) &
                (df_post_data['race_num'] == group), 'pred_labels'] = new_preds


  # evaluate the new predictions and save the scores
  # Write your code here.


KeyboardInterrupt: 

In [ ]:
# Your code for step 4 and 5


**Question**

Describe how you selected the best set of parameters. Furthermore, how do you interpret the best set of parameters that you found? And what do you think of the results of the new classifier?
>Answer

#### **Overall discussion**
For all 4 classifiers that you trained, describe:
- Does this classifier satisfies statistical parity?
- Does the classifier satisfy the equal opportunity criterion?

Finally, how do the different classifiers compare against each other?

>Answer

### **6. Intersectional fairness**
In the questions above `race` was the only protected attribute. However, multiple protected attributes sometimes interact, leading to different fairness outcomes for different combinations of these protected attributes.

Now explore the intersectional fairness for protected attributes `race` and `sex` for the first two classifiers from question 5. Make a combination of the `race` and `sex` column, resulting in four new subgroups (e.g., female Caucasian), and report the maximum difference between the subgroups for statistical parity, TPR and FPR.
For example, suppose we have four groups with TPRs 0.1, 0.2, 0.3, 0.8, then the maximum difference is 0.7.

Your code to evaluate intersectional fairness for Classifier 1:




In [ ]:
# Your code for intersectional fairness

Your code to evaluate intersectional fairness for Classifier 2:


In [ ]:
# Your code for intersectional fairness

**Question**

Write down a short interpretation of the results you calculated. What do you see?
> Answer:

## Discussion
Provide a short ethical discussion (1 or 2 paragraphs) reflecting on these two aspects:

1) The use of a ML system to try to predict recidivism;

2) The public release of a dataset like this.

> Answer